In [ ]:
#note always see document ur python version needs to below 3.11

In [3]:
pip install --upgrade pip setuptools wheel


   ---------------------------------------- 0.0/821.5 kB ? eta -:--:--
   ---------------------------------------- 10.2/821.5 kB ? eta -:--:--
   ---------------------------------------- 10.2/821.5 kB ? eta -:--:--
   ---------------------------------------- 10.2/821.5 kB ? eta -:--:--
   ---------------------------------------- 10.2/821.5 kB ? eta -:--:--
   ---------------------------------------- 10.2/821.5 kB ? eta -:--:--
   ----- -------------------------------- 122.9/821.5 kB 481.4 kB/s eta 0:00:02
   ----- -------------------------------- 122.9/821.5 kB 481.4 kB/s eta 0:00:02
   ----- -------------------------------- 122.9/821.5 kB 481.4 kB/s eta 0:00:02
   ----- -------------------------------- 122.9/821.5 kB 481.4 kB/s eta 0:00:02
   -------- ----------------------------- 194.6/821.5 kB 436.8 kB/s eta 0:00:02
   --------- ---------------------------- 204.8/821.5 kB 414.8 kB/s eta 0:00:02
   ---------- --------------------------- 225.3/821.5 kB 430.1 kB/s eta 0:00:02
   ------

In [1]:
!python --version


Python 3.8.18


In [3]:
pip install -U prefect

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import time
import joblib
import os
import prefect
from prefect import task, Flow
#from sklearn.model_selection import 

@task
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df.dropna()

@task
def preprocess_data(df):
    ratings_dict = {5.0: 1, 4.0: 1, 3.0: 0, 2.0: 0, 1.0: 0}
    df['sentiment'] = df['Ratings'].map(ratings_dict)
    x = df['Review text']
    y = df['sentiment']
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=50)

    def clean_text(text):
        text = re.sub(r"[^a-zA-Z]", " ", text)
        text = re.sub(r'\W+', ' ', text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = text.lower()
        stop_words = set(stopwords.words('english'))
        words = text.split()
        cleaned_words = [word for word in words if word not in stop_words]
        return ' '.join(cleaned_words)

    def lemmatize_text(text):
        lemmatizer = WordNetLemmatizer()
        tokens = nltk.word_tokenize(text)
        lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens]
        return ' '.join(lemmatized_words)

    x_train = x_train.apply(clean_text).apply(lemmatize_text)
    x_test = x_test.apply(clean_text).apply(lemmatize_text)
    
    return x_train, x_test, y_train, y_test

@task
def train_model(x_train, y_train):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('classifier', LogisticRegression())
    ])

    param_grid = {
        'tfidf__max_features': [1000, 2000, 3000],
        'classifier__C': [0.1, 1, 10],
        'classifier__penalty': ['l1', 'l2']
    }

    grid_search = GridSearchCV(estimator=pipeline,
                               param_grid=param_grid,
                               cv=5,
                               scoring='accuracy',
                               return_train_score=True,
                               verbose=1
                               )
    grid_search.fit(x_train, y_train)
    return grid_search.best_estimator_

@task
def evaluate_model(model, x_test, y_test):
    y_pred = model.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    return test_accuracy

with Flow("Sentiment Analysis") as flow:
    file_path = "data.csv"
    df = load_data(file_path)
    x_train, x_test, y_train, y_test = preprocess_data(df)
    model = train_model(x_train, y_train)
    test_accuracy = evaluate_model(model, x_test, y_test)

flow.run()


c:\Users\poppo\anaconda3\envs\mlproj\lib\site-packages\prefect\tasks.py:348: UserWarning: A task named 'load_data' and defined at 'C:\Users\poppo\AppData\Local\Temp\ipykernel_20524\732583359.py:20' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
c:\Users\poppo\anaconda3\envs\mlproj\lib\site-packages\prefect\tasks.py:348: UserWarning: A task named 'preprocess_data' and defined at 'C:\Users\poppo\AppData\Local\Temp\ipykernel_20524\732583359.py:25' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
c:\Users\poppo\anaconda3\envs\mlproj\lib\site-packages\prefect\tasks.py:348: UserWarning: A task named 'train_model' and defined at 'C:\Users\poppo\AppData\Local\Temp\ipykernel_20524\732583359.py:55' conflicts with another task. Consider specifying a unique `name` parameter in the task def

TypeError: 'fn' must be callable

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression  # Import LogisticRegression
import time
import joblib
import os
import prefect
from prefect import task, Flow

@task
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df.dropna()

@task
def preprocess_data(df):
    ratings_dict = {5.0: 1, 4.0: 1, 3.0: 0, 2.0: 0, 1.0: 0}
    df['sentiment'] = df['Ratings'].map(ratings_dict)
    x = df['Review text']
    y = df['sentiment']
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=50)

    def clean_text(text):
        text = re.sub(r"[^a-zA-Z]", " ", text)
        text = re.sub(r'\W+', ' ', text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = text.lower()
        stop_words = set(stopwords.words('english'))
        words = text.split()
        cleaned_words = [word for word in words if word not in stop_words]
        return ' '.join(cleaned_words)

    def lemmatize_text(text):
        lemmatizer = WordNetLemmatizer()
        tokens = nltk.word_tokenize(text)
        lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens]
        return ' '.join(lemmatized_words)

    x_train = x_train.apply(clean_text).apply(lemmatize_text)
    x_test = x_test.apply(clean_text).apply(lemmatize_text)
    
    return x_train, x_test, y_train, y_test

@task
def train_model(x_train, y_train):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('classifier', LogisticRegression())
    ])

    param_grid = {
        'tfidf__max_features': [1000, 2000, 3000],
        'classifier__C': [0.1, 1, 10],
        'classifier__penalty': ['l1', 'l2']
    }

    grid_search = GridSearchCV(estimator=pipeline,
                               param_grid=param_grid,
                               cv=5,
                               scoring='accuracy',
                               return_train_score=True,
                               verbose=1
                               )
    grid_search.fit(x_train, y_train)
    return grid_search.best_estimator_

@task
def evaluate_model(model, x_test, y_test):
    y_pred = model.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    return test_accuracy

with Flow("Sentiment Analysis") as flow:
    file_path = "data.csv"
    df = load_data(file_path)
    x_train, x_test, y_train, y_test = preprocess_data(df)
    model = train_model(x_train, y_train)
    test_accuracy = evaluate_model(model, x_test, y_test)

flow.run()


c:\Users\poppo\anaconda3\envs\mlproj\lib\site-packages\prefect\tasks.py:348: UserWarning: A task named 'load_data' and defined at 'C:\Users\poppo\AppData\Local\Temp\ipykernel_20524\1549570364.py:20' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
c:\Users\poppo\anaconda3\envs\mlproj\lib\site-packages\prefect\tasks.py:348: UserWarning: A task named 'preprocess_data' and defined at 'C:\Users\poppo\AppData\Local\Temp\ipykernel_20524\1549570364.py:25' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
c:\Users\poppo\anaconda3\envs\mlproj\lib\site-packages\prefect\tasks.py:348: UserWarning: A task named 'train_model' and defined at 'C:\Users\poppo\AppData\Local\Temp\ipykernel_20524\1549570364.py:55' conflicts with another task. Consider specifying a unique `name` parameter in the task 

TypeError: 'fn' must be callable